# **Tạo Span Adaptation Vector cho các argument (ARG) với chiến lược Inner Content Pooling**

## Mục đích

- Từ mỗi câu và các argument (ARG-0, ARG-1, ...) trong dữ liệu:
  - Kết hợp với adaptation vectors đã được trích xuất trước đó cho từng token.
  - Dùng mô hình SRL fine-tuned để lấy nhãn dự đoán per-token và trọng số classifier.
- Với mỗi argument, trên từng layer:
  - Lấy vector đầu (begin) và cuối (end) của span.
  - Tính vector nội dung (content vector) chỉ từ **các token bên trong** span (inner tokens, không gồm token đầu và cuối).
  - Ghép lại thành một Span Adaptation Vector: `[begin, end, content]`.
  - Lưu thêm trọng số của các inner tokens để có thể phân tích/khai thác sau.

## Đầu vào

1. Mô hình và tokenizer
   - Mô hình SRL đã fine-tuned (BioBERT):
     - Được load từ:
       - `Finetuned_Models/biobert-srl-best-model`
     - Dùng `AutoModelForTokenClassification`.
   - Tokenizer tương ứng:
     - Dùng `AutoTokenizer.from_pretrained(final_model_path)`.

2. Dữ liệu câu và argument (file JSON)
   - Nằm trong các thư mục:
     - `Clean_Dataset/Corpus/Split_GramVar/Train`
     - `Clean_Dataset/Corpus/Split_GramVar/Test`
     - `Clean_Dataset/Corpus/Split_ParaVE/Train`
     - `Clean_Dataset/Corpus/Split_ParaVE/Test`
   - Mỗi file JSON:
     - Chứa một list các phần tử, mỗi phần tử có:
       - `text`: câu gốc (string).
       - `arguments`: dict, ví dụ:
         - `{"ARG-0": "...", "ARG-1": "...", ...}`.

3. Adaptation vectors cho từng câu
   - Nằm trong các thư mục:
     - `adaptation_vectors_train_gramvar_pt_aligned`
     - `adaptation_vectors_test_gramvar_pt_aligned`
     - `adaptation_vectors_train_parave_pt_aligned`
     - `adaptation_vectors_test_parave_pt_aligned`
   - Mỗi câu tương ứng một file:
     - `sentence_i.pt` bên trong thư mục động từ:
       - `.../<av_input_dir>/<verb_name>/sentence_<i>.pt`
   - Mỗi file `.pt` là một dict:
     - `{ "layer_0": tensor[num_tokens, hidden_dim], "layer_1": tensor[num_tokens, hidden_dim], ... }`.

4. Thiết bị tính toán
   - Tự động chọn:
     - GPU (CUDA) nếu có.
     - CPU nếu không có GPU.

## Đầu ra

1. Span Adaptation Vectors cho từng argument
   - Được lưu trong các thư mục:
     - `span_adaptation_vectors_train_gramvar_inner_content`
     - `span_adaptation_vectors_test_gramvar_inner_content`
     - `span_adaptation_vectors_train_parave_inner_content`
     - `span_adaptation_vectors_test_parave_inner_content`
   - Bên trong mỗi thư mục trên:
     - Có các thư mục con theo động từ: `<verb_name>/`.

2. Cấu trúc từng file `.pt` đầu ra
   - Tên file:
     - `sentence_<i>_<ARG_x>.pt`, ví dụ:
       - `sentence_0_ARG_0.pt`
       - `sentence_5_ARG_1.pt`
   - Nội dung file:
     - Dict với cấu trúc:
       - `layer_k`: span_vec (tensor) có dạng `[hidden_dim * 3]` (ghép `begin`, `end`, `content_vec`).
       - `layer_k_token_weights`: tensor chứa các trọng số `a_k` (hoặc rỗng) tương ứng với inner tokens trên layer đó.

## Quy trình

### 1. Thiết lập môi trường và tải mô hình

- Import các thư viện: `torch`, `os`, `glob`, `json`, `tqdm`, `transformers`, `numpy`.
- Chọn thiết bị:
  - `device = torch.device("cuda" if torch.cuda.is_available() else "cpu")`.
- Khai báo đường dẫn gốc:
  - `drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'`.
- Load mô hình và tokenizer:
  - `final_model_path = os.path.join(drive_base_path, 'Finetuned_Models/biobert-srl-best-model')`.
  - `tokenizer = AutoTokenizer.from_pretrained(final_model_path)`.
  - `model_ft = AutoModelForTokenClassification.from_pretrained(final_model_path).to(device)`.
  - `model_ft.eval()`.

### 2. Khai báo danh sách các bộ dữ liệu cần xử lý

- `datasets_to_process` là một list các tuple:
  - `(name, data_dir, av_input_dir, output_dir)`.
- Ví dụ một phần:
  - `("Train_GramVar", <data_dir_train_gramvar>, <av_input_dir_train_gramvar>, <output_dir_train_gramvar_vectors_inner>)`.
- Mỗi tuple tương ứng với:
  - Tên bộ dữ liệu (Train/Test + GramVar/ParaVE).
  - Thư mục dữ liệu JSON.
  - Thư mục chứa adaptation vectors.
  - Thư mục output để lưu span adaptation vectors.

### 3. Hàm `find_sub_list(sl, l)`

- Chức năng:
  - Tìm vị trí `(start_idx, end_idx)` của list con `sl` trong list `l`.
- Ứng dụng:
  - `sl` = tokenized argument (`arg_tokens`).
  - `l`  = tokenized full sentence (`full_tokens`).
  - Trả về `(-1, -1)` nếu không tìm thấy.

### 4. Hàm `create_span_vectors(sentence_text, arguments, token_av_data, model, tokenizer)`

- Mục đích:
  - Tạo Span Adaptation Vectors cho tất cả các argument trong một câu, với Inner Content Pooling.

- Các bước chi tiết:

1. Tokenize câu và chạy mô hình:
   - `full_tokens = tokenizer.tokenize(sentence_text)`.
   - Với `torch.no_grad()`:
     - Tạo `inputs = tokenizer(sentence_text, return_tensors="pt").to(device)`.
     - Chạy mô hình: `logits = model(**inputs).logits`.
     - Lấy `predictions = argmax(logits, dim=2)` (nhãn dự đoán cho từng token).
   - Lấy `classifier_weights = model.classifier.weight.detach().cpu()`.

2. Duyệt từng argument trong `arguments`:
   - Chỉ xét các key bắt đầu bằng `"ARG"` và có `arg_text` không rỗng.
   - Tokenize `arg_text` → `arg_tokens`.
   - Gọi `find_sub_list(arg_tokens, full_tokens)` để tìm `(start_idx, end_idx)`.
   - Cộng thêm 1 cho cả hai index:
     - Do input mô hình có `[CLS]` ở đầu.
   - Nếu `start_idx == 0` (trường hợp không hợp lệ) thì bỏ qua.

3. Xử lý trên từng layer trong `token_av_data`:
   - `layer_av_tensor = layer_av_tensor.cpu()` để đưa về CPU.
   - Lấy vector begin và end:
     - `adap_vec_begin = layer_av_tensor[start_idx]`.
     - `adap_vec_end = layer_av_tensor[end_idx]`.

4. Xác định inner tokens (token nội dung):
   - `content_start_idx = start_idx + 1`.
   - `content_end_idx = end_idx`.
   - Nếu `content_start_idx < content_end_idx`:
     - Có inner tokens (span dài > 2 token):
       - `inner_span_token_avs = layer_av_tensor[content_start_idx : content_end_idx]`.
       - `inner_span_token_preds = predictions[content_start_idx : content_end_idx]`.
   - Ngược lại:
     - Không có inner tokens (span dài 1 hoặc 2 token):
       - `inner_span_token_avs` là tensor rỗng.
       - `inner_span_token_preds` là array rỗng.

5. Tính các hệ số `a_k` cho inner tokens:
   - Khởi tạo `a_k_list = []`.
   - Với mỗi inner token:
     - `adap_vec_tk = inner_span_token_avs[i]`.
     - `pred_label_id = inner_span_token_preds[i]`.
     - `w_r_star = classifier_weights[pred_label_id]`.
     - Tính:
       - `dot_product = torch.dot(adap_vec_tk, w_r_star)`.
       - `norm_w = torch.linalg.norm(w_r_star)`.
       - `a_k = max(0, dot_product / (norm_w + 1e-8))`.
     - Thêm `a_k` vào `a_k_list`.
   - Nếu có `a_k_list`:
     - `stacked_a_k = torch.stack(a_k_list)`.
   - Ngược lại:
     - `stacked_a_k = torch.empty(0)` (tensor rỗng).
   - `sum_a_k = torch.sum(stacked_a_k) + 1e-8`.

6. Tính content vector từ inner tokens:
   - Khởi tạo:
     - `content_vec = torch.zeros(layer_av_tensor.shape[1], dtype=torch.float32)`.
   - Nếu `sum_a_k > 1e-8` và có ít nhất một inner token:
     - Với mỗi inner token:
       - `w_k = a_k_list[i] / sum_a_k`.
       - Cộng dồn:
         - `content_vec += w_k * inner_span_token_avs[i]`.
   - Nếu không có inner tokens hoặc `sum_a_k` gần 0:
     - `content_vec` giữ nguyên là vector 0 (không đóng góp nội dung).

7. Ghép span vector và lưu kết quả:
   - `span_vec = torch.cat([adap_vec_begin, adap_vec_end, content_vec])`.
   - Lưu:
     - `span_vectors_per_layer[layer_name] = span_vec`.
     - `span_vectors_per_layer[f"{layer_name}_token_weights"] = stacked_a_k` (trọng số inner tokens).

8. Lưu per-argument:
   - `span_vectors_all_args[arg_label] = span_vectors_per_layer`.

- Hàm trả về:
  - `span_vectors_all_args`:
    - Dict: `{ "ARG-0": {layer_0: span_vec, layer_0_token_weights: ..., ...}, "ARG-1": {...}, ... }`.

### 5. Vòng lặp chính trong `if __name__ == "__main__":`

- Dùng để chạy toàn bộ quy trình cho tất cả các bộ dữ liệu.

1. In thông báo bắt đầu.
2. Với mỗi `(name, data_dir, av_input_dir, output_dir)` trong `datasets_to_process`:
   - Tạo `output_dir` nếu chưa tồn tại.
   - Lấy danh sách các file JSON trong `data_dir`.
   - Nếu không tìm thấy file nào:
     - In cảnh báo và bỏ qua bộ dữ liệu đó.
   - Tạo `tqdm` để hiển thị tiến độ theo file.

3. Với mỗi file JSON:
   - Suy ra `verb_name` từ tên file (bỏ `_train_set.json` hoặc `_test_set.json`).
   - Đọc dữ liệu:
     - `original_data = json.load(f)`.
   - Tạo thư mục:
     - `output_verb_dir = os.path.join(output_dir, verb_name)`.

4. Với mỗi câu trong `original_data` (theo index `i`):
   - Lấy:
     - `sentence_text = item.get('text')`.
     - `arguments = item.get('arguments')`.
   - Nếu thiếu `sentence_text` hoặc `arguments`: bỏ qua.
   - Tạo đường dẫn tới adaptation vectors:
     - `av_path = os.path.join(av_input_dir, verb_name, f"sentence_{i}.pt")`.
   - Load:
     - `token_av_data = torch.load(av_path)`.

   - Gọi:
     - `span_av_data_per_arg = create_span_vectors(sentence_text, arguments, token_av_data, model_ft, tokenizer)`.

   - Nếu có kết quả:
     - Với mỗi `arg_label, vectors_for_this_arg`:
       - Tạo `safe_arg_label = arg_label.replace('-', '_')`.
       - Tạo `output_filename = f"sentence_{i}_{safe_arg_label}.pt"`.
       - Tạo `output_path = os.path.join(output_verb_dir, output_filename)`.
       - Lưu file bằng `torch.save(vectors_for_this_arg, output_path)`.

5. Xử lý lỗi:
   - `FileNotFoundError`:
     - Bỏ qua câu đó (không in lỗi).
   - Các lỗi khác:
     - In thông báo qua `pbar.write(...)` với chỉ số câu và tên động từ.
   - Cập nhật progress bar sau mỗi file JSON.

6. In thông báo hoàn tất sau khi xử lý xong toàn bộ.


In [ ]:
# Import các thư viện cần thiết
import torch
import os
import glob
import json
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForTokenClassification
import numpy as np

# THIẾT LẬP CÁC ĐƯỜNG DẪN VÀ TẢI MÔ HÌNH

# Tự động chọn GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Tải lại mô hình fine-tuned để lấy trọng số classifier và chạy dự đoán
print("Đang tải lại mô hình Fine-tuned...")
final_model_path = os.path.join(drive_base_path, 'Finetuned_Models/biobert-srl-best-model')
tokenizer = AutoTokenizer.from_pretrained(final_model_path)
model_ft = AutoModelForTokenClassification.from_pretrained(final_model_path).to(device)
model_ft.eval() # Chuyển sang chế độ đánh giá
print(" -> Tải mô hình thành công!")

# --- DANH SÁCH CÁC BỘ DỮ LIỆU CẦN XỬ LÝ ---
datasets_to_process = [
    # --- Dữ liệu Train ---
    (
        "Train_GramVar",
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Train'),
        os.path.join(drive_base_path, 'Adaptation Vector/Train/adaptation_vectors_train_gramvar_pt_aligned'),
        os.path.join(drive_base_path, 'span_adaptation_vectors_train_gramvar_inner_content')
    ),
    (
        "Train_ParaVE",
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Train'),
        os.path.join(drive_base_path, 'Adaptation Vector/Train/adaptation_vectors_train_parave_pt_aligned'),
        os.path.join(drive_base_path, 'span_adaptation_vectors_train_parave_inner_content')
    ),
    # --- Dữ liệu Test ---
    (
        "Test_GramVar",
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Test'),
        os.path.join(drive_base_path, 'Adaptation Vector/Test/adaptation_vectors_test_gramvar_pt_aligned'),
        os.path.join(drive_base_path, 'span_adaptation_vectors_test_gramvar_inner_content')
    ),
    (
        "Test_ParaVE",
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Test'),
        os.path.join(drive_base_path, 'Adaptation Vector/Train/adaptation_vectors_test_parave_pt_aligned'),
        os.path.join(drive_base_path, 'span_adaptation_vectors_test_parave_inner_content')
    ),
]

# CÁC HÀM XỬ LÝ CỐT LÕI

def find_sub_list(sl, l):
    sll = len(sl)
    for ind in (i for i, e in enumerate(l) if e == sl[0]):
        if l[ind:ind+sll] == sl:
            return ind, ind + sll - 1
    return -1, -1

def create_span_vectors(sentence_text, arguments, token_av_data, model, tokenizer):
    """
    Tạo span adaptation vector cho tất cả các argument trong một câu.
    *** ĐÃ CẬP NHẬT: Content pooling chỉ tính các token BÊN TRONG (inner tokens) ***
    """
    full_tokens = tokenizer.tokenize(sentence_text)

    with torch.no_grad():
        inputs = tokenizer(sentence_text, return_tensors="pt").to(device)
        logits = model(**inputs).logits
        predictions = torch.argmax(logits, dim=2)[0].cpu().numpy()

    classifier_weights = model.classifier.weight.detach().cpu()

    span_vectors_all_args = {}

    for arg_label, arg_text in arguments.items():
        if not arg_label.startswith('ARG') or not arg_text:
            continue

        arg_tokens = tokenizer.tokenize(arg_text)
        start_idx, end_idx = find_sub_list(arg_tokens, full_tokens)

        start_idx += 1
        end_idx += 1

        if start_idx == 0:
            continue

        span_vectors_per_layer = {}
        for layer_name, layer_av_tensor in token_av_data.items():
            layer_av_tensor = layer_av_tensor.cpu()

            # Lấy vector đầu và cuối
            adap_vec_begin = layer_av_tensor[start_idx]
            adap_vec_end = layer_av_tensor[end_idx]

            # Xác định các token NỘI DUNG (ở giữa)
            content_start_idx = start_idx + 1
            content_end_idx = end_idx # Slicing (vd: [a:b]) sẽ không bao gồm b

            # Lấy vector và dự đoán của các token NỘI DUNG
            if content_start_idx < content_end_idx:
                # Có token ở giữa (cho span dài > 2)
                inner_span_token_avs = layer_av_tensor[content_start_idx : content_end_idx]
                inner_span_token_preds = predictions[content_start_idx : content_end_idx]
            else:
                # Không có token ở giữa (cho span dài 1 hoặc 2)
                inner_span_token_avs = torch.empty(0, layer_av_tensor.shape[1], dtype=layer_av_tensor.dtype) # Tensor rỗng
                inner_span_token_preds = np.array([]) # Array rỗng

            # Tính trọng số a_k CHỈ cho các token NỘI DUNG
            a_k_list = []
            for i in range(len(inner_span_token_avs)): # Vòng lặp này sẽ bỏ qua nếu span rỗng
                adap_vec_tk = inner_span_token_avs[i]
                pred_label_id = inner_span_token_preds[i]
                w_r_star = classifier_weights[pred_label_id]

                dot_product = torch.dot(adap_vec_tk, w_r_star)
                norm_w = torch.linalg.norm(w_r_star)
                a_k = torch.max(torch.tensor(0.0), dot_product / (norm_w + 1e-8))
                a_k_list.append(a_k)

            stacked_a_k = torch.stack(a_k_list) if a_k_list else torch.empty(0)
            sum_a_k = torch.sum(stacked_a_k) + 1e-8

            # Tính content_vec CHỈ từ các token NỘI DUNG
            content_vec = torch.zeros(layer_av_tensor.shape[1], dtype=torch.float32)
            if sum_a_k > 1e-8 and len(inner_span_token_avs) > 0:
                for i in range(len(inner_span_token_avs)):
                    w_k = a_k_list[i] / sum_a_k
                    content_vec += w_k * inner_span_token_avs[i]

            # Ghép vector
            span_vec = torch.cat([adap_vec_begin, adap_vec_end, content_vec])
            span_vectors_per_layer[layer_name] = span_vec

            # Lưu trọng số (trọng số của token NỘI DUNG)
            span_vectors_per_layer[f"{layer_name}_token_weights"] = stacked_a_k

        span_vectors_all_args[arg_label] = span_vectors_per_layer

    return span_vectors_all_args

# THỰC THI CHƯƠNG TRÌNH

if __name__ == "__main__":
    print("\n--- Bắt đầu quá trình tạo Span Adaptation Vector (Inner Content Pooling) ---")

    for name, data_dir, av_input_dir, output_dir in datasets_to_process:
        print(f"\n=================================================")
        print(f"BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: {name}")
        print(f" -> Output Dir: {output_dir}")
        print(f"=================================================")

        os.makedirs(output_dir, exist_ok=True)
        json_files = glob.glob(os.path.join(data_dir, '*.json'))

        if not json_files:
            print(f"CẢNH BÁO: Không tìm thấy file dữ liệu gốc nào trong '{data_dir}'. Bỏ qua.")
            continue

        pbar = tqdm(total=len(json_files), desc=f"Xử lý {name}", unit="file")

        for json_file in json_files:
            verb_name = os.path.basename(json_file).replace('_test_set.json', '').replace('_train_set.json', '')

            with open(json_file, 'r', encoding='utf-8') as f:
                original_data = json.load(f)

            output_verb_dir = os.path.join(output_dir, verb_name)
            os.makedirs(output_verb_dir, exist_ok=True)

            for i, item in enumerate(original_data):
                sentence_text = item.get('text')
                arguments = item.get('arguments')

                if not sentence_text or not arguments:
                    continue

                try:
                    av_path = os.path.join(av_input_dir, verb_name, f"sentence_{i}.pt")
                    token_av_data = torch.load(av_path)

                    span_av_data_per_arg = create_span_vectors(sentence_text, arguments, token_av_data, model_ft, tokenizer)

                    if span_av_data_per_arg:
                        for arg_label, vectors_for_this_arg in span_av_data_per_arg.items():
                            safe_arg_label = arg_label.replace('-', '_')
                            output_filename = f"sentence_{i}_{safe_arg_label}.pt"
                            output_path = os.path.join(output_verb_dir, output_filename)

                            torch.save(vectors_for_this_arg, output_path)

                except FileNotFoundError:
                    pass
                except Exception as e:
                    pbar.write(f"Lỗi khi xử lý câu {i}, động từ {verb_name}: {e}")

            pbar.update(1)
        pbar.close()

    print("\n--- QUÁ TRÌNH TẠO SPAN ADAPTATION VECTOR (Inner Content Pooling) ĐÃ HOÀN TẤT! ---")

Sử dụng thiết bị: cuda
Đang tải lại mô hình Fine-tuned...
 -> Tải mô hình thành công!

--- Bắt đầu quá trình tạo Span Adaptation Vector (Inner Content Pooling) ---

BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: Train_GramVar
 -> Output Dir: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_vectors_train_gramvar_inner_content


Xử lý Train_GramVar:   0%|          | 0/35 [00:00<?, ?file/s]


BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: Train_ParaVE
 -> Output Dir: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_vectors_train_parave_inner_content


Xử lý Train_ParaVE:   0%|          | 0/35 [00:00<?, ?file/s]


BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: Test_GramVar
 -> Output Dir: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_vectors_test_gramvar_inner_content


Xử lý Test_GramVar:   0%|          | 0/35 [00:00<?, ?file/s]


BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: Test_ParaVE
 -> Output Dir: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_vectors_test_parave_inner_content


Xử lý Test_ParaVE:   0%|          | 0/35 [00:00<?, ?file/s]


--- QUÁ TRÌNH TẠO SPAN ADAPTATION VECTOR (Inner Content Pooling) ĐÃ HOÀN TẤT! ---
